<a href="https://colab.research.google.com/github/Jayati23/Keras_NLP/blob/master/simple_universal_sentence_encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import  numpy  as  np
import  tensorflow  as tf
import tensorflow_hub as hub
import csv
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import drive
import re
drive.mount('/content/gdrive')
import nltk


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
with open("/content/gdrive/My Drive/Colab Notebooks/quora.tsv") as tsvfile:
    tsvreader = csv.reader(tsvfile, delimiter="\t")
   
    quest1 = []
    quest2 = []
    dup = []
    for line in tsvreader:
        quest1.append(line[3])
        quest2.append(line[4])
        dup.append(line[5])
        


In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/1"
embed = hub.Module(module_url)

In [0]:
def remove_stopwords(stop_words, tokens):
    res = []
    for token in tokens:
        if not token in stop_words:
            res.append(token)
 
    return res

def process_text(text):
 #   text = text.encode('ascii', errors='ignore').decode()
    text = text.lower()
    text = re.sub(r'http\S+', ' ', text)
    text = re.sub(r'#+', ' ', text )
    text = re.sub(r'@[A-Za-z0-9]+', ' ', text)
    text = re.sub(r"([A-Za-z]+)'s", r"\1 is", text)
    #text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"won't", "will not ", text)
    text = re.sub(r"isn't", "is not ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub(r'\d+', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip()
    return text

def lemmatize(tokens):
    lemmatizer = nltk.stem.WordNetLemmatizer()
    lemma_list = []
    for token in tokens:
        lemma = lemmatizer.lemmatize(token, 'v')
        if lemma == token:
            lemma = lemmatizer.lemmatize(token)
        lemma_list.append(lemma)
    # return [ lemmatizer.lemmatize(token, 'v') for token in tokens ]
    return lemma_list


def process_all(text):
    text = process_text(text)
    stop_words = ['.',',','!', 'ourselves', 'hers','between','yourself', 'but', 'again', 'there', 'about', 'once','during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself','other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i','after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than']
    return ' '.join(remove_stopwords(stop_words, text.split()))


In [0]:
def cosine_similarity(v1, v2):
    mag1 = np.linalg.norm(v1)
    mag2 = np.linalg.norm(v2)
    if (not mag1) or (not mag2):
        return 0
    return np.dot(v1, v2) / (mag1 * mag2)

In [0]:
def get_features(texts):
    if type(texts) is str:
        texts = [texts]
    with tf.Session() as sess:
        sess.run([tf.global_variables_initializer(), tf.tables_initializer()])
        return sess.run(embed(texts))
    

In [0]:
def test_similarity(text1, text2):
    vec1 = get_features(text1)[0]
    vec2 = get_features(text2)[0]
    return cosine_similarity(vec1, vec2)

In [0]:
quest1_final = [] 
quest2_final = [] 

for i in range(len(quest1)):
  quest1_final.append(process_text(quest1[i]))
  quest2_final.append(process_text(quest2[i]))
  

In [0]:
count = 0
for i in range(1,10):
  sent1 = quest1_final[i]
  sent2 = quest2_final[i]
  print(sent1, sent2)
  k = test_similarity(sent1,sent2)
  print(k, dup[i])
  if(k >= 0.85 and dup[i] == '1'):
    count = count +1
  elif(k < 0.85 and dup[i] == '0'):
    count = count + 1
    
    
print(count)
    

what is the step by step guide to invest in share market in india what is the step by step guide to invest in share market
0.9638024 0
what is the story of kohinoor koh i noor diamond what would happen if the indian government stole the kohinoor koh i noor diamond back
0.63465446 0
how can i increase the speed of my internet connection while using a vpn how can internet speed be increased by hacking through dns
0.89566684 0
why am i mentally very lonely how can i solve it find the remainder when math math is divided by
0.31800973 0
which one dissolve in water quikly sugar salt methane and carbon di oxide which fish would survive in salt water
0.52574474 0
astrology i am a capricorn sun cap moon and cap rising what does that say about me i am a triple capricorn sun moon and ascendant in capricorn what does this say about me
0.9356637 1
should i buy tiago what keeps childern active and far from phone and video games
0.019359237 0
how can i be a good geologist what should i do to be a gre